In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import hamming_loss, f1_score, roc_auc_score,accuracy_score, classification_report, ConfusionMatrixDisplay, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from skmultilearn.model_selection import IterativeStratification
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from model3 import ManualLogisticRegressionOneVsRest
from scipy.sparse import hstack
import joblib

In [2]:
DATA_TRAIN_DIR = "../../ML_Approaching/Preprocessing/train_data_preprocessing.csv"
DATA_VAL_DIR = "../../ML_Approaching/Preprocessing/val_data_preprocessing.csv"
DATA_TEST_DIR = "../../ML_Approaching/Preprocessing/test_data_preprocessing.csv"

In [3]:
def load_data(file_path):

    df = pd.read_csv(file_path)
    
    # 3. Định nghĩa các cột nhãn
    label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    
    X = df['clean_text'].astype(str)
    
    y = df[label_cols]
    
    return X, y

# --- THỰC HIỆN LOAD DỮ LIỆU ---
print("Đang load dữ liệu...")

X_train, y_train = load_data(DATA_TRAIN_DIR)
X_val, y_val     = load_data(DATA_VAL_DIR)
X_test, y_test   = load_data(DATA_TEST_DIR)

Đang load dữ liệu...


In [4]:
tfidf_vectorizer = TfidfVectorizer(
    strip_accents='unicode',
    ngram_range=(1, 2),       # giữ ngữ cảnh "not good", "you are"
    min_df=3,                 # lọc nhiễu nhẹ
    max_df=0.9,               # loại từ quá phổ biến
    max_features=50000,
)

In [5]:
tfidf_vectorizer.fit(X_train)

TfidfVectorizer(max_df=0.9, max_features=50000, min_df=3, ngram_range=(1, 2),
                strip_accents='unicode')

In [6]:
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [7]:
tfidf_vectorizer.get_feature_names_out()[:100]

array(['__', '__ __', '__toc__', '_friend', '_noticeboard',
       '_noticeboard incidents', 'aa', 'aa aa', 'aah', 'aaliyah', 'aap',
       'aardvark', 'aaron', 'ab', 'aba', 'abandon', 'abandoned',
       'abandoning', 'abandonment', 'abbas', 'abbey', 'abbreviated',
       'abbreviation', 'abbreviations', 'abc', 'abc news', 'abd',
       'abduction', 'abdul', 'abdullah', 'abe', 'abhira', 'abhishek',
       'abhorrent', 'abide', 'abiding', 'abilities', 'ability',
       'ability create', 'ability customize', 'ability edit',
       'ability rename', 'ability upload', 'abit', 'abject', 'abkhazia',
       'able', 'able add', 'able block', 'able change', 'able come',
       'able contribute', 'able determine', 'able edit', 'able find',
       'able get', 'able help', 'able keep', 'able make', 'able post',
       'able provide', 'able read', 'able see', 'able stand', 'able take',
       'able tell', 'able understand', 'able use', 'able work',
       'able write', 'abnormal', 'aboard', 'aboli

In [8]:
print("TF-IDF shape (train):", X_train_tfidf.shape)
print("TF-IDF shape (val):  ", X_val_tfidf.shape)
print("TF-IDF shape (test):", X_test_tfidf.shape)

TF-IDF shape (train): (130038, 50000)
TF-IDF shape (val):   (15958, 50000)
TF-IDF shape (test): (15958, 50000)


In [9]:
def exact_match_ratio(y_true, y_pred):
    return np.mean(np.all(y_true == y_pred, axis=1))

In [10]:
def evaluate_model(model, X_test, y_test):
    # Predict
    y_pred_proba = model.predict_proba(X_test)   # shape (N, L)
    y_pred = (y_pred_proba > 0.5).astype(int)    # threshold = 0.5

    results = {
        "ROC-AUC": roc_auc_score(
            y_test,
            y_pred_proba,
            average="macro"     # hoặc "micro"
        ),
        "F1_Macro": f1_score(
            y_test,
            y_pred,
            average="macro"
        ),
        "Hamming_Loss": hamming_loss(
            y_test,
            y_pred
        ),
        "EMR": accuracy_score(
            y_test,
            y_pred
        )
    }

    return pd.DataFrame([results])

In [11]:
log_model = ManualLogisticRegressionOneVsRest(learning_rate=1.0,n_iters=1000,lambda_param=0.1)
log_model.fit(X_train_tfidf,y_train)

Bắt đầu huấn luyện 6 mô hình Binary với lr=1.0, lambda=0.1...


ManualLogisticRegressionOneVsRest(lambda_param=0.1, learning_rate=1.0)

In [12]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
log_results = evaluate_model(log_model, X_test_tfidf, y_test)

print("=== Logistic Regression Performance ===")
display(log_results)
print("Mean ROC-AUC:", log_results["ROC-AUC"])

=== Logistic Regression Performance ===


,ROC-AUC,F1_Macro,Hamming_Loss,EMR
0,0.965703,0.475761,0.03787,0.873042


Mean ROC-AUC: 0    0.965703
Name: ROC-AUC, dtype: float64


In [13]:
def find_optimal_thresholds(y_true, y_prob):
    """
    Tìm ngưỡng tối ưu cho từng nhãn dựa trên F1-score
    (Đã fix lỗi KeyError do Pandas DataFrame)
    """
    # 1. ÉP KIỂU VỀ NUMPY ARRAY ĐỂ TRÁNH LỖI DATAFRAME SLICING
    # Dù đầu vào là DataFrame, List hay Array thì dòng này đều xử lý được
    y_true = np.array(y_true)
    y_prob = np.array(y_prob)
    
    best_thresholds = []
    thresholds = np.arange(0.01, 0.9, 0.005) # Quét từ 0.1 đến 0.9
    
    n_labels = y_true.shape[1]
    
    print(f"Đang tìm ngưỡng cho {n_labels} nhãn...")
    
    for i in range(n_labels):
        y_t = y_true[:, i]
        y_p = y_prob[:, i]
        
        best_score = -1
        best_thresh = 0.5
        
        # Kiểm tra nếu cột nhãn toàn là 0 (ví dụ tập Val không có mẫu threat nào)
        if np.sum(y_t) == 0:
            print(f"Label {i}: Không có mẫu Positive nào trong tập Val. Giữ nguyên threshold 0.5")
            best_thresholds.append(0.5)
            continue

        for thresh in thresholds:
            pred_binary = (y_p > thresh).astype(int)
            score = f1_score(y_t, pred_binary)
            
            if score > best_score:
                best_score = score
                best_thresh = thresh
                
        best_thresholds.append(best_thresh)
        print(f"Label {i}: Best Threshold = {best_thresh:.2f}, F1-Score = {best_score:.4f}")
        
    return best_thresholds
y_val_proba = log_model.predict_proba(X_val_tfidf)
# --- CHẠY LẠI ---
# Không cần sửa code gọi hàm, chỉ cần chạy lại cell này
print("Bắt đầu tìm ngưỡng...")
best_thresholds = find_optimal_thresholds(y_val, y_val_proba)
print("\nDanh sách ngưỡng tối ưu:", best_thresholds)

Bắt đầu tìm ngưỡng...
Đang tìm ngưỡng cho 6 nhãn...
Label 0: Best Threshold = 0.53, F1-Score = 0.6972
Label 1: Best Threshold = 0.73, F1-Score = 0.4000
Label 2: Best Threshold = 0.52, F1-Score = 0.7435
Label 3: Best Threshold = 0.84, F1-Score = 0.4490
Label 4: Best Threshold = 0.52, F1-Score = 0.6694
Label 5: Best Threshold = 0.69, F1-Score = 0.3395

Danh sách ngưỡng tối ưu: [np.float64(0.5349999999999999), np.float64(0.7349999999999999), np.float64(0.5249999999999999), np.float64(0.8399999999999999), np.float64(0.5249999999999999), np.float64(0.695)]


In [14]:
from sklearn.metrics import f1_score, roc_auc_score, hamming_loss, accuracy_score
import pandas as pd
import numpy as np

def evaluate_with_thresholds(model, X_test, y_test, thresholds):
    """
    Đánh giá mô hình với danh sách ngưỡng (thresholds) riêng cho từng nhãn.
    Trả về DataFrame chứa: ROC-AUC, F1-Samples, Hamming Loss, EMR.
    """
    # 1. Dự đoán xác suất
    print("Đang dự đoán xác suất trên tập Test...")
    y_prob = model.predict_proba(X_test)
    
    # 2. Ép kiểu về Numpy Array (quan trọng để tránh lỗi)
    y_test_np = np.array(y_test)
    y_prob_np = np.array(y_prob)
    
    # 3. Áp dụng ngưỡng (Thresholding)
    # Tạo ma trận dự đoán nhị phân (0/1) dựa trên ngưỡng từng cột
    y_pred = np.zeros_like(y_prob_np)
    
    print("Đang áp dụng ngưỡng tối ưu...")
    for i in range(y_prob_np.shape[1]):
        # Nếu xác suất >= ngưỡng của nhãn i -> gán bằng 1, ngược lại 0
        y_pred[:, i] = (y_prob_np[:, i] >= thresholds[i]).astype(int)
        
    # 4. Tính toán các chỉ số
    print("Đang tính toán metrics...")
    
    # ROC-AUC: Tính trên xác suất (y_prob), không phụ thuộc threshold
    # average='macro': Tính trung bình cộng AUC của các nhãn
    roc_auc = roc_auc_score(y_test_np, y_prob_np, average='macro')
    
    # F1-Samples: Tính F1 cho từng nhãn rồi lấy trung bình
    f1_samples = f1_score(y_test_np, y_pred, average='macro')
    
    # Hamming Loss: Tỷ lệ các nhãn bị dự đoán sai trên tổng số nhãn
    # (Càng thấp càng tốt)
    h_loss = hamming_loss(y_test_np, y_pred)
    
    # Exact Match Ratio (EMR): Tỷ lệ các mẫu được dự đoán đúng HOÀN TOÀN cả 6 nhãn
    # (Trong sklearn, accuracy_score cho multi-label chính là EMR)
    emr = exact_match_ratio(y_test_np, y_pred)
    
    # 5. Đóng gói kết quả
    results = {
        "Metric": ["ROC-AUC", "F1 Score (Samples)", "Hamming Loss", "Exact Match Ratio"],
        "Value": [roc_auc, f1_samples, h_loss, emr]
    }
    
    return pd.DataFrame(results)

# --- CÁCH SỬ DỤNG ---
# Giả sử bạn đã có best_thresholds từ bước trước
# Và X_test_tfidf, y_test đã sẵn sàng

df_results = evaluate_with_thresholds(log_model, X_test_tfidf, y_test, best_thresholds)

print("\n=== KẾT QUẢ ĐÁNH GIÁ CUỐI CÙNG ===")
print(df_results)

Đang dự đoán xác suất trên tập Test...
Đang áp dụng ngưỡng tối ưu...
Đang tính toán metrics...

=== KẾT QUẢ ĐÁNH GIÁ CUỐI CÙNG ===
               Metric     Value
0             ROC-AUC  0.965703
1  F1 Score (Samples)  0.534331
2        Hamming Loss  0.025139
3   Exact Match Ratio  0.901805


# Tinh chỉnh Logistic Regression và TF IDF

In [16]:
param_grid = {
    # Tinh chỉnh TF-IDF
    'tfidf__max_features': [50000],
    'tfidf__ngram_range': [(1, 2), (1,3)],
    'tfidf__min_df': [1,3,5],
    'tfidf__max_df': [0.8, 0.9],

    # Tinh chỉnh Model Multi-label
    'clf__learning_rate': [1.0, 0.1],
    'clf__lambda_param': [1.0, 0.1, 0.01],
    'clf__n_iters': [1000]
}

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(strip_accents='unicode')),
    ('clf', ManualLogisticRegressionOneVsRest())
])

cv_strategy = IterativeStratification(n_splits=3, order=1)

roc_auc_score_macro = make_scorer(
    roc_auc_score,
    needs_proba=True,
    average="macro"
)

grid = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=cv_strategy, # Bắt buộc phải set cái này
    scoring=roc_auc_score_macro, # Tương đương ROC-AUC Macro
    verbose=3,
    n_jobs=-1
)

print("Bắt đầu GridSearch Multi-label...")
grid.fit(X_train,y_train)


print("\n--- KẾT QUẢ ---")
print(f"Best ROC_AUC_MACRO: {grid.best_score_:.4f}")
print("Best Params:", grid.best_params_)

Bắt đầu GridSearch Multi-label...
Fitting 3 folds for each of 72 candidates, totalling 216 fits


e:\GitHub\CS221_NLP_Jigsaw_toxic_comment_classification_project\hf_env\Lib\site-packages\sklearn\model_selection\_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Bắt đầu huấn luyện 6 mô hình Binary với lr=1.0, lambda=1.0...

--- KẾT QUẢ ---
Best ROC_AUC_MACRO: nan
Best Params: {'clf__lambda_param': 1.0, 'clf__learning_rate': 1.0, 'clf__n_iters': 1000, 'tfidf__max_df': 0.8, 'tfidf__max_features': 50000, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 2)}


In [19]:
best_model = grid.best_estimator_
log_results = evaluate_with_thresholds(best_model, X_test, y_test, best_thresholds)

print("=== Logistic Regression (Tuned) Performance ===")
display(log_results)

Đang dự đoán xác suất trên tập Test...
Đang áp dụng ngưỡng tối ưu...
Đang tính toán metrics...
=== Logistic Regression (Tuned) Performance ===


,Metric,Value
0,ROC-AUC,0.965626
1,F1 Score (Samples),0.535580
2,Hamming Loss,0.025128
3,Exact Match Ratio,0.901679


In [20]:
joblib.dump(best_model, 'best_logistic_custom_model.joblib')
print("Đã lưu mô hình thành công!")

Đã lưu mô hình thành công!


In [ ]:
# Lưu thresholds tối ưu để sử dụng trong production
joblib.dump(best_thresholds, 'optimal_thresholds.joblib')
print(f"\nĐã lưu optimal thresholds: {best_thresholds}")
print("\nThông tin chi tiết:")
for i, label in enumerate(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']):
    print(f"  {label}: {best_thresholds[i]:.2f}")

In [42]:
import numpy as np
import pandas as pd

def analyze_sample_strict_format(model_ovr, text_input, vectorizer, true_labels_vec, optimal_thresholds):
    """
    In ra phân tích chi tiết theo format yêu cầu:
    Câu -> Từ lọc -> Chi tiết từng nhãn (Bảng tính W*x -> Logit -> Prob -> Kết luận).
    """
    # ==========================================================================
    # 1. CÂU BAN ĐẦU & CÁC NHÃN TƯƠNG ỨNG
    # ==========================================================================
    # Lấy tên các nhãn thực tế
    label_names_all = model_ovr.labels
    true_active_labels = [label_names_all[i] for i, val in enumerate(true_labels_vec) if val == 1]
    
    # Nếu không có nhãn nào (mẫu sạch)
    if not true_active_labels:
        true_lbl_str = "KHÔNG CÓ (Mẫu sạch/Clean)"
    else:
        true_lbl_str = ", ".join([l.upper() for l in true_active_labels])

    print("\n" + "#" * 100)
    print(f"📌 CÂU BAN ĐẦU: \"{str(text_input).strip()}\"")
    print(f"👉 CÁC LABEL TƯƠNG ỨNG THỰC TẾ: [{true_lbl_str}]")
    print("-" * 100)

    # ==========================================================================
    # 2. DANH SÁCH CÁC TỪ SÀNG LỌC & GIÁ TRỊ TF-IDF
    # ==========================================================================
    x_vec = vectorizer.transform([text_input])
    feature_indices = x_vec.indices
    tfidf_values = x_vec.data
    feature_names = vectorizer.get_feature_names_out()
    
    print("DATA INPUT: DANH SÁCH CÁC TỪ ĐƯỢC LỌC KÈM GIÁ TRỊ TF-IDF:")
    if len(feature_indices) == 0:
        print("   (Không tìm thấy từ nào trong từ điển)")
    else:
        # In ra dạng list dễ nhìn
        words_display = [f"'{feature_names[i]}': {v:.4f}" for i, v in zip(feature_indices, tfidf_values)]
        print("   " + " | ".join(words_display))
    
    print("=" * 100)

    # ==========================================================================
    # 3. PHÂN TÍCH CHI TIẾT TỪNG NHÃN (LÀM ĐỀU HẾT CÁC NHÃN)
    # ==========================================================================
    
    for i, label_name in enumerate(label_names_all):
        sub_model = model_ovr.models[i]
        threshold = optimal_thresholds[i]
        bias = sub_model.bias
        weights = sub_model.weights
        
        print(f"\n🏷️  PHÂN TÍCH NHÃN: {label_name.upper()}")
        print(f"   {'-'*75}")
        print(f"   {'TỪ (TOKEN)':<20} | {'TRỌNG SỐ (W)':<15} | {'TF-IDF (x)':<15} | {'W * x':<15}")
        print(f"   {'-'*75}")
        
        z_sum_words = 0
        
        # Duyệt qua từng từ để in bảng tính
        for idx, tfidf_val in zip(feature_indices, tfidf_values):
            word = feature_names[idx]
            w_val = weights[idx]
            contribution = w_val * tfidf_val
            z_sum_words += contribution
            
            # In dòng chi tiết
            print(f"   {word:<20} | {w_val:<15.4f} | {tfidf_val:<15.4f} | {contribution:<15.4f}")
            
        print(f"   {'-'*75}")
        
        # Tính toán Logit & Xác suất
        total_logit = bias + z_sum_words
        prob = 1 / (1 + np.exp(-total_logit))
        
        # Kết luận
        pred_val = 1 if prob > threshold else 0
        true_val = true_labels_vec[i]
        match_str = "✅ ĐÚNG" if pred_val == true_val else "❌ SAI"
        
        print(f"   ► Bias (Hệ số tự do) : {bias:.4f}")
        print(f"   ► Tổng đóng góp từ   : {z_sum_words:.4f}")
        print(f"   ------------------------------")
        print(f"   ► GIÁ TRỊ LOGIT (z)  : {total_logit:.4f}")
        print(f"   ► XÁC SUẤT (P)       : {prob:.4f}")
        print(f"   ► SO SÁNH NGƯỠNG     : {prob:.4f} {'LỚN HƠN' if prob > threshold else 'NHỎ HƠN'} {threshold:.4f}")
        print(f"   ► KẾT LUẬN           : Dự đoán = {pred_val} | Thực tế = {true_val} -> {match_str}")
    
    print("\n" + "="*100 + "\n")

# ==============================================================================
# PHẦN CHẠY: CHỌN 3 MẪU ĐÚNG VÀ 3 MẪU SAI ĐỂ IN
# ==============================================================================

# Lấy các thành phần
best_pipeline = grid.best_estimator_
vectorizer_final = best_pipeline.named_steps['tfidf']
clf_final = best_pipeline.named_steps['clf']

y_test_vals = y_test.values if hasattr(y_test, 'values') else np.array(y_test)
# Reset index cho X_test để khớp vị trí với numpy array
X_test_reset = X_test.reset_index(drop=True) if hasattr(X_test, 'reset_index') else X_test

# 2. Tính toán lại dự đoán & trạng thái Đúng/Sai
y_prob_run = best_pipeline.predict_proba(X_test)
y_pred_run = np.zeros_like(y_prob_run)
for i in range(len(opt_thresholds)):
    y_pred_run[:, i] = (y_prob_run[:, i] > opt_thresholds[i]).astype(int)

# Check đúng sai cho từng mẫu (Đúng là phải khớp cả 6 nhãn)
is_correct_run = np.all(y_test_vals == y_pred_run, axis=1)

# 3. LỌC RIÊNG CÁC MẪU "THÚ VỊ" (CÓ NHÃN)

# --- MẪU ĐÚNG VÀ CÓ NHÃN (Toxic Correct) ---
# Điều kiện: (Đúng hoàn toàn) VÀ (Tổng nhãn thực tế > 0)
correct_toxic_mask = is_correct_run & (np.sum(y_test_vals, axis=1) > 0)
correct_toxic_indices = np.where(correct_toxic_mask)[0]

# --- MẪU SAI (Như cũ) ---
incorrect_indices = np.where(~is_correct_run)[0]

print(f"📊 TÌM THẤY TRONG TẬP TEST:")
print(f"- Số mẫu Sạch (Clean) dự đoán đúng: {np.sum(is_correct_run & (np.sum(y_test_vals, axis=1) == 0))}")
print(f"- Số mẫu Độc hại (Toxic) dự đoán đúng: {len(correct_toxic_indices)} <--- Ta sẽ in cái này")
print(f"- Số mẫu dự đoán sai: {len(incorrect_indices)}")

# ==============================================================================
# TIẾN HÀNH IN THEO FORMAT BẠN YÊU CẦU
# ==============================================================================

# --- 1. IN 3 MẪU ĐÚNG (NHƯNG PHẢI LÀ MẪU CÓ NHÃN ĐỘC HẠI) ---
print("\n" + "★"*40 + " 3 MẪU ĐÚNG (CÓ NHÃN TOXIC) " + "★"*40)

if len(correct_toxic_indices) == 0:
    print("⚠️ Không tìm thấy mẫu nào có nhãn Toxic mà mô hình đoán đúng cả! (Model quá yếu hoặc Threshold quá cao)")
else:
    # Lấy 3 mẫu đầu tiên tìm được
    for idx in correct_toxic_indices[:3]:
        analyze_sample_strict_format(
            model_ovr=clf_final,
            text_input=X_test_reset.iloc[idx] if hasattr(X_test_reset, 'iloc') else X_test_reset[idx],
            vectorizer=vectorizer_final,
            true_labels_vec=y_test_vals[idx],
            optimal_thresholds=best_thresholds
        )

# --- 2. IN 3 MẪU SAI (ƯU TIÊN FALSE NEGATIVE ĐỂ PHÂN TÍCH) ---
print("\n" + "★"*40 + " 3 MẪU SAI (ĐA DẠNG) " + "★"*40)

diverse_errors = []

# Loại 1: Sót lọt (Nguy hiểm nhất) - Thực tế có nhãn, nhưng máy đoán toàn 0
fn_indices = np.where((np.sum(y_test_vals, axis=1) > 0) & (np.sum(y_pred_run, axis=1) == 0))[0]
if len(fn_indices) > 0: diverse_errors.append(fn_indices[0])

# Loại 2: Báo động giả - Thực tế sạch, nhưng máy đoán có nhãn
fp_indices = np.where((np.sum(y_test_vals, axis=1) == 0) & (np.sum(y_pred_run, axis=1) > 0))[0]
if len(fp_indices) > 0: diverse_errors.append(fp_indices[0])

# Loại 3: Sai khác (Sai 1 phần) - Có nhãn, máy cũng đoán có nhãn, nhưng không khớp nhau hoàn toàn
partial_error_mask = (np.sum(y_test_vals, axis=1) > 0) & (np.sum(y_pred_run, axis=1) > 0) & (~is_correct_run)
partial_indices = np.where(partial_error_mask)[0]
if len(partial_indices) > 0: diverse_errors.append(partial_indices[0])

# Nếu không đủ 3 loại trên thì lấy thêm mẫu sai ngẫu nhiên
if len(diverse_errors) < 3:
    remaining = [i for i in incorrect_indices if i not in diverse_errors]
    diverse_errors.extend(remaining[:3-len(diverse_errors)])

for idx in diverse_errors:
    analyze_sample_strict_format(
        model_ovr=clf_final,
        text_input=X_test_reset.iloc[idx] if hasattr(X_test_reset, 'iloc') else X_test_reset[idx],
        vectorizer=vectorizer_final,
        true_labels_vec=y_test_vals[idx],
        optimal_thresholds=best_thresholds
    )

📊 TÌM THẤY TRONG TẬP TEST:
- Số mẫu Sạch (Clean) dự đoán đúng: 14038
- Số mẫu Độc hại (Toxic) dự đoán đúng: 355 <--- Ta sẽ in cái này
- Số mẫu dự đoán sai: 1565

★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★ 3 MẪU ĐÚNG (CÓ NHÃN TOXIC) ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★

####################################################################################################
📌 CÂU BAN ĐẦU: "tony sidaway obviously fistfuckee loves arm ass"
👉 CÁC LABEL TƯƠNG ỨNG THỰC TẾ: [TOXIC, OBSCENE, INSULT]
----------------------------------------------------------------------------------------------------
DATA INPUT: DANH SÁCH CÁC TỪ ĐƯỢC LỌC KÈM GIÁ TRỊ TF-IDF:
   'arm': 0.4063 | 'ass': 0.2882 | 'loves': 0.3877 | 'obviously': 0.2653 | 'sidaway': 0.4426 | 'tony': 0.3693 | 'tony sidaway': 0.4460

🏷️  PHÂN TÍCH NHÃN: TOXIC
   ---------------------------------------------------------------------------
   TỪ (TOKEN)           | TRỌNG SỐ (W)    | TF-IDF (x)      | W * x          
   -------------------------

In [50]:
best_model.named_steps["tfidf"].vocabulary_.__sizeof__()

61516528